In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install openpyxl

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models
from openpyxl import Workbook

In [4]:
path_data = "/content/drive/MyDrive/Hybrid_DL_Kcr/data/"
path_model = "/content/drive/MyDrive/Hybrid_DL_Kcr/Model/"
path_result = "/content/drive/MyDrive/Hybrid_DL_Kcr/Model/"

In [5]:
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [6]:
# Train data
file_train = "train_data.csv"
df_train =pd.read_csv(path_data+file_train, delimiter= ',')

texts_train =[] #PTMsequend kmer
for i in df_train['Sequence']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer train data
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])

Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape

(13950, 2)

In [7]:
# load test data
file_test ="test_data.csv"
df_test =pd.read_csv(path_data+file_test,delimiter= ',')
text_test =[] #PTMsequend kmer
for i in df_test['Sequence']:
  temp = ProSentence(i,k)
  text_test.append(temp)
df_test['k_mer'] =text_test

test_sequences = []
for each in text_test:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences.append(each_index_list)

Xtest = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytest = np.array(df_test['Label'])
# perform one-hot encoding on the labels
ytest = np.array(ytest)
lb = LabelBinarizer()
ytest= lb.fit_transform(ytest)
ytest = to_categorical(ytest)
ytest.shape

(5978, 2)

In [8]:
from tensorflow.keras import models, layers, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Concatenate, Embedding
from tensorflow.keras.layers import Dropout, BatchNormalization, Dense, Bidirectional, LSTM

from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, BatchNormalization, Flatten, Dense
from tensorflow.keras.models import Model

def create_model(vocab_size, max_sequence_length):
    inputs = Input(shape=(max_sequence_length,))
    embedding = Embedding(vocab_size + 1, 300, input_length=max_sequence_length, trainable=True)(inputs)

    # Bi-LSTM branch only
    lstm = Bidirectional(LSTM(32, return_sequences=True))(embedding)
    lstm = Dropout(0.4)(lstm)
    lstm = BatchNormalization()(lstm)
    lstm = Flatten()(lstm)  # Chuyển đầu ra Bi-LSTM thành vector

    # Fully Connected Layers
    dense1 = Dense(64, activation='relu')(lstm)
    dense1 = Dropout(0.4)(dense1)
    dense1 = BatchNormalization()(dense1)

    outputs = Dense(2, activation='softmax')(dense1)  # Binary classification

    model = Model(inputs, outputs)
    return model




**Distillation học trên model teacher với bộ dl Generic**

In [9]:
from keras import models
from tensorflow.keras.optimizers import AdamW
model= create_model(vocab_size, MAX_SEQUENCE_LENGTH)
# Compile the model
model.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])
model.fit(Xtrain, ytrain,
              epochs=50,
              batch_size =16,
              )
model.save(path_model + "Bi-LSTM.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6492 - loss: 0.7013
Epoch 2/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7561 - loss: 0.5127
Epoch 3/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7855 - loss: 0.4669
Epoch 4/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7978 - loss: 0.4418
Epoch 5/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8104 - loss: 0.4225
Epoch 6/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8195 - loss: 0.4119
Epoch 7/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8279 - loss: 0.3956
Epoch 8/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8280 - loss: 0.3999
Epoch 9/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8330 - loss: 0.3841
Epoch 10/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8350 - loss: 0.3783
Epoch 11/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8376 - loss: 0.3679
Epoch 12/50
872/872 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/ste

In [10]:
import numpy as np
import json
import os  # Import os to check if the path exists
import pandas as pd  # Import pandas for easy CSV saving
from sklearn.metrics import confusion_matrix  # Import confusion_matrix


ypred = model.predict(Xtest)

# Ensure predictions are in the correct format
ypred = np.argmax(ypred, axis=1)  # Getting the predicted class
ytest_true = np.argmax(ytest, axis=1)  # Getting the true class

# Compute the confusion matrix
confusion_mat = confusion_matrix(ytest_true, ypred)  # Rename to avoid overwriting

# Calculate TP, TN, FP, FN
TN = confusion_mat[0, 0]  # True Negative
FP = confusion_mat[0, 1]  # False Positive
FN = confusion_mat[1, 0]  # False Negative
TP = confusion_mat[1, 1]  # True Positive

# Create a DataFrame to hold TP, TN, FP, FN values
data = {'TP': [TP],
        'FP': [FP],
        'TN': [TN],
        'FN': [FN]}

df = pd.DataFrame(data)

output_file = 'Bi-LSTM.xlsx'

# Write the DataFrame to an Excel file
df.to_excel(os.path.join(path_result, output_file), index=False)

print(f"Các giá trị TP, TN, FP, FN đã được ghi vào file {output_file} dưới dạng cột.")


187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Các giá trị TP, TN, FP, FN đã được ghi vào file Bi-LSTM.xlsx dưới dạng cột.


In [11]:
#Cross-validation
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
# Khởi tạo KFold với k=10
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)


# Khởi tạo các biến lưu trữ kết quả cross-validation
all_metrics = []
confusion_matrices = []

# Thực hiện K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")

    # Tạo dữ liệu train và validation cho fold hiện tại
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]

    model= create_model(vocab_size, MAX_SEQUENCE_LENGTH)
    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train_fold, y_train_fold,
                  epochs=50,
                  batch_size =16,
                  )

    # Lưu lại kết quả cho từng fold
    results = model.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)

    # Dự đoán nhãn cho tập validation
    y_pred_fold = model.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # Tính confusion matrix và lưu TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Tính trung bình kết quả từ các fold
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(model.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Hiển thị các giá trị TP, TN, FP, FN cho từng fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# Hiển thị và lưu các giá trị TP, TN, FP, FN cho từng fold vào file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_Bi-LSTM.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_Bi-LSTM.xlsx'")

Training on fold 1/10...
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6140 - loss: 0.7768
Epoch 2/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7318 - loss: 0.5400
Epoch 3/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7727 - loss: 0.4951
Epoch 4/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7858 - loss: 0.4605
Epoch 5/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8121 - loss: 0.4216
Epoch 6/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8182 - loss: 0.4196
Epoch 7/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8224 - loss: 0.4020
Epoch 8/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8375 - loss: 0.3857
Epoch 9/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8274 - loss: 0.3934
Epoch 10/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8342 - loss: 0.3771
Epoch 11/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8385 - loss: 0.3712
Epoch 12/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accurac

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6348 - loss: 0.7490
Epoch 2/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7388 - loss: 0.5259
Epoch 3/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7677 - loss: 0.4951
Epoch 4/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7931 - loss: 0.4505
Epoch 5/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8001 - loss: 0.4378
Epoch 6/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8189 - loss: 0.4099
Epoch 7/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8189 - loss: 0.4139
Epoch 8/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8314 - loss: 0.3954
Epoch 9/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8254 - loss: 0.3901
Epoch 10/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8381 - loss: 0.3756
Epoch 11/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8417 - loss: 0.3730
Epoch 12/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accurac

...

785/785 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6281 - loss: 0.7229
Epoch 2/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7241 - loss: 0.5364
Epoch 3/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7619 - loss: 0.4962
Epoch 4/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7882 - loss: 0.4543
Epoch 5/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8074 - loss: 0.4264
Epoch 6/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8074 - loss: 0.4196
Epoch 7/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8194 - loss: 0.4051
Epoch 8/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8190 - loss: 0.4007
Epoch 9/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8420 - loss: 0.3729
Epoch 10/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8400 - loss: 0.3731
Epoch 11/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8328 - loss: 0.3751
Epoch 12/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accurac

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6317 - loss: 0.7584
Epoch 2/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7235 - loss: 0.5502
Epoch 3/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7583 - loss: 0.5028
Epoch 4/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7892 - loss: 0.4545
Epoch 5/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8039 - loss: 0.4288
Epoch 6/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8045 - loss: 0.4345
Epoch 7/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8256 - loss: 0.3997
Epoch 8/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8299 - loss: 0.3917
Epoch 9/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8337 - loss: 0.3822
Epoch 10/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8361 - loss: 0.3820
Epoch 11/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8495 - loss: 0.3555
Epoch 12/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accurac

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


785/785 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.6341 - loss: 0.7334
Epoch 2/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7274 - loss: 0.5404
Epoch 3/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7768 - loss: 0.4789
Epoch 4/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7907 - loss: 0.4564
Epoch 5/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8089 - loss: 0.4354
Epoch 6/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8059 - loss: 0.4265
Epoch 7/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8112 - loss: 0.4145
Epoch 8/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8229 - loss: 0.4042
Epoch 9/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8253 - loss: 0.3956
Epoch 10/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8385 - loss: 0.3724
Epoch 11/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8397 - loss: 0.3706
Epoch 12/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accurac